In [3]:
from pathlib import Path
from astropy.io import fits
from astropy.table import Table
import shutil


source=Path("/home/nicolas/Documents/Research/PhD/JWST-Data/PRISM/Spectra1D/")

mast_folder=source / "MAST"
jades_folder=source / "JADES"


# By ID

This search in both folders that any of the files contains the same sequence of numbers

In [ ]:
obj_ID="6002"

print("Example 1: \n")
for file in jades_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)
    
for file in mast_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)    



obj_ID="5113"

print("Example 2: \n")

for file in jades_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)
    
for file in mast_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)    
        


Example 1: 

hlsp_jades_jwst_nirspec_goods-s-deephst-00006002_clear-prism_v1.0_x1d.fits
Example 1: 

hlsp_jades_jwst_nirspec_goods-s-deephst-10005113_clear-prism_v1.0_x1d.fits
jw01210-o001_s000005113_nirspec_clear-prism_x1d.fits


This extracts the obj_ID from JADES folder files, but only works for the ones starting in 000

In [3]:
import re

for file in jades_folder.iterdir():
    match=re.search(r"100(\d+)", file.name)
    if match:
        ID = match.group(1)
        print(ID)

05469
15338
58975
12477
37201
05113
14220
00183
08071
40620
36262
35328
13597
00098
12081
01587
11955
08722
00633
13393
12005
36834
05447
14117
09609
13704
01568
13774
35295
00367
13545
73
01283
03811
11378
40
00796
38972
12284
04398
06084
09506
00814
12552
13682
13609
08231
12655
09382
12344
01946
37047
12566
15193
10642
11388
00966
10639
13578
09848
15344
11974
12488
13385
14177
00081
01843
00625
09320
03788
16442
12669
09693
12616
10770
11987
05217
12194
10691
11980
00626
00219
01761
56849
36017
11849
03750
13620
04819
04721
40868
13618
12493
00338
13797
16186
01697
00618
16374
00085
01916
13905
01892
00483
00478
14170
08067
12170
00160
05367
04061
12511


Let's now combine them: extract all the IDs from JADEs and then search for coincidences in MAST files

In [4]:
import re
import numpy as np

#Extract IDs and plug them into array
IDs_array = []


for file in jades_folder.iterdir():
    match=re.search(r"deephst-100(\d+)", file.name)
    if match:
        obj_ID = int(match.group(1))
        IDs_array.append(obj_ID)

        
IDs_array.sort()

#Print files that contain ID in their names

jades_files=[]
mast_files=[]

for obj_ID in IDs_array:
    for file in jades_folder.iterdir():
        if str(obj_ID) in file.name:
            jades_files.append(file.name)
        
    for file in mast_folder.iterdir():
        if str(obj_ID) in file.name:
            mast_files.append(file.name)    
            
#Compare lengths

print(len(IDs_array),
len(jades_files),
len(mast_files) )


110 140 151


I don't know why there are more entries in jades_files than in IDs_array, they should be the same size. Let's just extract the ID and then us the array to make the plots with a loop

In [11]:
import re
import numpy as np
import matplotlib.pyplot as plt



#Extract IDs and plug them into array
IDs_array = []


for file in jades_folder.iterdir():
    match=re.search(r"deephst-100(\d+)", file.name)
    if match:
        obj_ID = match.group(1)
        IDs_array.append(obj_ID)     

 

This is the function that we will use in the loop, it plots the spectra

In [26]:
def plot_spectrum(MAST_file, JADES_file, ID):
  
    
    # ~ fits.open('file.fits')
    t_MAST=Table.read(MAST_file, hdu="EXTRACT1D")
    w_MAST=t_MAST['WAVELENGTH']
    flux_MAST=t_MAST['FLUX'] #flux in Jy
    #z=1
    #rest_w=w/(1+z) #in micras
    #rest_w_Angstrom=rest_w*1e-6/1e-10 # rest wavelength in Angstrom
    w_Angstrom=w_MAST*1e-6/1e-10 # observed wavelength in Angstrom
    flux_ergs = 2.99792458E-05 *flux_MAST / w_Angstrom**2. #flux in erg s^-1 cm^-2 A^-1



    #archivo de JADES
    t_JADES=Table.read(JADES_file, hdu="EXTRACT1D")
    w_JADES=t_JADES['WAVELENGTH']
    flux_JADES=t_JADES['FLUX'] #flux in Jy

    
    plt.rcParams["figure.figsize"] = (8, 6)
    plt.title(f'ID {ID}, z=', fontsize=18.)
    
    plt.step(w_MAST/1e-4,flux_ergs, label='MAST Data-Archive', color='Blue')
    plt.step(w_JADES/1e-4,flux_JADES, label='JADES Survey', color='Green')
    plt.xlabel(r'$\lambda_{obs}$ [$\AA$]', fontsize=22)
    plt.ylabel(r'FLUX [erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]', fontsize=22)
    legend=plt.legend(loc=4,labelspacing=0.1)
    legend=plt.legend(loc='best',labelspacing=0.1)
    plt.setp(legend.get_texts(),fontsize='14')
    plt.savefig(f"PRISM_Plots/{ID}.pdf")
    plt.close()
    

Test of the loop

In [ ]:
for ID in ["00367", "01587", "01697"]:
    mast_file = './Spectra1D/MAST/jw01210-o001_s0000'+ID+'_nirspec_clear-prism_x1d.fits'
    jades_file = './Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-100'+ID+'_clear-prism_v1.0_x1d.fits'

    plot_spectrum(mast_file, jades_file, ID)

Finally, just change the array [---] for IDs_array computed previously

# By Coordinates

Now, we want to read the coordinates from the header of the .fits files


In [16]:
## Let's open the .fits file
hdulist=fits.open(jades_folder / 'hlsp_jades_jwst_nirspec_goods-s-deephst-00002332_clear-prism_v1.0_x1d.fits')# Abrimos imagen
hdulist.info() # Para ver informacion de la imagen en la terminal
#hdulist[0].header.keys()# Ver los headers y su contenido

RA_jades = hdulist[0].header['RA'] # Ver algun header en especifico (Right assention)
DEC_jades = hdulist[0].header['DEC'] # Ver algun header en especifico (Right assention)
print ('RA =', RA_jades)
print ('DEC =', DEC_jades)

Filename: /home/nicolas/Documents/Research/PhD/JWST-Data/PRISM/Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-00002332_clear-prism_v1.0_x1d.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      34   ()      
  1  EXTRACT1D     1 BinTableHDU     21   674R x 3C   [D, D, D]   
  2  ASDF          1 BinTableHDU      9   0R x 0C   []   
RA = 53.15724
DEC = -27.82377


Same but for MAST files

In [21]:
hdulist=fits.open(mast_folder / 'jw01210-o001_s000005115_nirspec_clear-prism_x1d.fits')# Abrimos imagen
hdulist.info() # Para ver informacion de la imagen en la terminal
#hdulist[0].header.keys()# Ver los headers y su contenido

RA_jades = hdulist[0].header['TARG_RA'] # Ver algun header en especifico (Right assention)
DEC_jades = hdulist[0].header['TARG_DEC'] # Ver algun header en especifico (Right assention)
print ('RA =', RA_jades)
print ('DEC =', DEC_jades)

Filename: /home/nicolas/Documents/Research/PhD/JWST-Data/PRISM/Spectra1D/MAST/jw01210-o001_s000005115_nirspec_clear-prism_x1d.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     281   ()      
  1  EXTRACT1D     1 BinTableHDU     90   432R x 18C   [D, D, D, D, D, D, D, D, D, D, D, J, D, D, D, D, D, D]   
  2  ASDF          1 BinTableHDU     11   1R x 1C   [140813B]   
RA = 53.150504166666664
DEC = -27.7908611111111


Let's print this data for the files whose ID number coincide in the file name, to check how different they are. First, we define a function that returns a table with JADES' RA, DEC, and MAST's RA and DED:

In [11]:
def Print_Coordinates(mast_file, jades_file):
    
    hdulist_mast=fits.open(mast_file)
    
    RA_mast=hdulist_mast[0].header['TARG_RA']
    DEC_mast=hdulist_mast[0].header['TARG_DEC']
    
    hdulist_jades=fits.open(jades_file)
    
    RA_jades=hdulist_jades[0].header['RA']
    DEC_jades=hdulist_jades[0].header['DEC']
    
    print("ID\t", "RA_mast", '\t', "RA_jades", '|', "DEC_mast", '\t', "DEC_jades")
    print(ID,"\t", RA_mast, '\t', RA_jades, '|', DEC_mast, '\t', DEC_jades)
       
    

Lates make a quick example:

In [12]:
ID='05113'
mast_file = './Spectra1D/MAST/jw01210-o001_s0000'+ID+'_nirspec_clear-prism_x1d.fits'
jades_file = './Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-100'+ID+'_clear-prism_v1.0_x1d.fits'

Print_Coordinates(mast_file, jades_file)

ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
05113 	 53.150504166666664 	 53.1673 | -27.7908611111111 	 -27.80287


Now let's make an example with the loop:

In [15]:
for ID in ["00367", "01587", "01697"]:
    mast_file = './Spectra1D/MAST/jw01210-o001_s0000'+ID+'_nirspec_clear-prism_x1d.fits'
    jades_file = './Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-100'+ID+'_clear-prism_v1.0_x1d.fits'

    Print_Coordinates(mast_file, jades_file)

ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
00367 	 53.150504166666664 	 53.15231 | -27.7908611111111 	 -27.81778
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
01587 	 53.150504166666664 	 53.17134 | -27.7908611111111 	 -27.80157
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
01697 	 53.150504166666664 	 53.16032 | -27.7908611111111 	 -27.80036


In [17]:
import re


#Extract IDs and plug them into array
IDs_array = []


for file in jades_folder.iterdir():
    match=re.search(r"deephst-100(\d+)", file.name)
    if match:
        obj_ID = match.group(1)
        IDs_array.append(obj_ID)     
   
        
for ID in IDs_array:
    mast_file = './Spectra1D/MAST/jw01210-o001_s0000'+ID+'_nirspec_clear-prism_x1d.fits'
    jades_file = './Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-100'+ID+'_clear-prism_v1.0_x1d.fits'

    Print_Coordinates(mast_file, jades_file)

ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
05469 	 53.150504166666664 	 53.17338 | -27.7908611111111 	 -27.77609
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
15338 	 53.150504166666664 	 53.11535 | -27.7908611111111 	 -27.77289
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
58975 	 53.150504166666664 	 53.11243 | -27.7908611111111 	 -27.77461
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
12477 	 53.150504166666664 	 53.12258 | -27.7908611111111 	 -27.76373
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
37201 	 53.150504166666664 	 53.10892 | -27.7908611111111 	 -27.7951
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
05113 	 53.150504166666664 	 53.1673 | -27.7908611111111 	 -27.80287
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
14220 	 53.150504166666664 	 53.16477 | -27.7908611111111 	 -27.77463
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
00183 	 53.150504166666664 	 53.15557 | -27.7908611111111 	 -27.82177
ID	 RA_mast 	 RA_jades | DEC_mast 	 DEC_jades
08071 	 53.150504166666664 	

It works, sadly, all MAST files have same RA and DEC.